In [23]:
import pandas as pd
import geopandas as gpd

In [24]:
path_rilevazioni_pm2_5 = f'../dataset/rilevazioni/rilevazioni_PM2_5.csv'
path_rilevazioni_pm10 = f'../dataset/rilevazioni/rilevazioni_PM10.csv'
path_comuni = f'../dataset/dati_spaziali/comuni/comuni.shp'

In [25]:
stazioni = pd.DataFrame(columns= ['nome', 'lat', 'long', 'id_comune'])
rileva = pd.DataFrame(columns= ['nome_stazione', 'sigla_indicatore'])
rilevazioni = pd.DataFrame(columns= ['data_rilevazione', 'valore', 'sigla_indicatore', 'nome_stazione'])
comuni = gpd.read_file(path_comuni)

In [26]:
rilevazioni_pm2_5 = pd.read_csv(path_rilevazioni_pm2_5, 
                                sep=';', 
                                skiprows=[0, 1, 2, 3 , 4, 5, 6, 8],
                                skipfooter=1,
                                engine='python', 
                                index_col=False)
rilevazioni_pm10 = pd.read_csv(path_rilevazioni_pm10,
                                sep=';', 
                                skiprows=[0, 1, 2, 3, 4, 5, 7],
                                skipfooter=1,
                                engine='python', 
                                index_col=False)

In [27]:
rilevazioni_pm2_5.head()

,NomeCentralina,DataRilevazione,Provincia,Comune,Sigla,Valore,Superamenti PM10*
0,Altamura - Via Santeramo,2019-01-01 00:00:00.0,Bari,Altamura,PM2.5,7,NaN
1,Altamura - Via Santeramo,2019-01-02 00:00:00.0,Bari,Altamura,PM2.5,8,NaN
2,Altamura - Via Santeramo,2019-01-03 00:00:00.0,Bari,Altamura,PM2.5,4,NaN
3,Altamura - Via Santeramo,2019-01-04 00:00:00.0,Bari,Altamura,PM2.5,6,NaN
4,Altamura - Via Santeramo,2019-01-05 00:00:00.0,Bari,Altamura,PM2.5,8,NaN


In [28]:
rilevazioni_pm10.head()

,NomeCentralina,DataRilevazione,Provincia,Comune,Sigla,Valore,Superamenti PM10*
0,Altamura - Via Santeramo,2019-01-01 00:00:00.0,Bari,Altamura,PM10,11,7
1,Altamura - Via Santeramo,2019-01-02 00:00:00.0,Bari,Altamura,PM10,11,7
2,Altamura - Via Santeramo,2019-01-03 00:00:00.0,Bari,Altamura,PM10,6,7
3,Altamura - Via Santeramo,2019-01-04 00:00:00.0,Bari,Altamura,PM10,8,7
4,Altamura - Via Santeramo,2019-01-05 00:00:00.0,Bari,Altamura,PM10,9,7


In [7]:
columns = ['Nome', 'Data', 'Comune', 'Sigla', 'Valore']
rilevazioni_pm2_5 = rilevazioni_pm2_5[['NomeCentralina ', 'DataRilevazione ', 'Comune ', 'Sigla ', 'Valore ']]
rilevazioni_pm2_5.columns = columns

rilevazioni_pm10 = rilevazioni_pm10[['NomeCentralina ', 'DataRilevazione ', 'Comune ', 'Sigla ', 'Valore ']]
rilevazioni_pm10.columns = columns

Nei file csv delle rilevazioni, al cambio della centralina viene ripetuto l'header, l'inquinante e la provincia. Vengono quindi rimosse queste righe

In [8]:
rilevazioni_pm2_5 = rilevazioni_pm2_5[rilevazioni_pm2_5["Nome"].str.contains('Provincia:|NomeCentralina|Inquinante:') == False]
rilevazioni_pm2_5.reset_index(inplace=True, drop=True)
rilevazioni_pm10 = rilevazioni_pm10[rilevazioni_pm10["Nome"].str.contains('Provincia:|NomeCentralina|Inquinante:') == False]
rilevazioni_pm10.reset_index(inplace=True, drop=True)

rilevazioni_pm2_5["Nome"] = rilevazioni_pm2_5["Nome"].apply(lambda x: x.split("-")[1].strip().replace("  ", " "))
rilevazioni_pm10["Nome"] = rilevazioni_pm10["Nome"].apply(lambda x: x.split("-")[1].strip().replace("  ", " "))

Estrazione della latitudine e longitudine delle stazioni

In [9]:
import requests
import json

In [10]:
def get_lat_long(stazione, comune):
    s = stazione.replace(' ', '+')
    c = comune.replace(' ', '+')
    request = requests.get(f'https://nominatim.openstreetmap.org/search?country=Italia&city={comune}&street={stazione}&format=jsonv2&limit=1')
    json_object = json.loads(request.content)
    if len(json_object) == 0:
        request = requests.get(f'https://nominatim.openstreetmap.org/search?country=Italia&city={comune}&format=jsonv2&limit=1')
        json_object = json.loads(request.content)
    return json_object[0]['lat'], json_object[0]['lon']

In [11]:
rilevazioni_pm2_5.head()

,Nome,Data,Comune,Sigla,Valore
0,Via Santeramo,2019-01-01 00:00:00.0,Altamura,PM2.5,7
1,Via Santeramo,2019-01-02 00:00:00.0,Altamura,PM2.5,8
2,Via Santeramo,2019-01-03 00:00:00.0,Altamura,PM2.5,4
3,Via Santeramo,2019-01-04 00:00:00.0,Altamura,PM2.5,6
4,Via Santeramo,2019-01-05 00:00:00.0,Altamura,PM2.5,8


In [12]:
stazioni_pm2_5_comuni_nomi = set(zip(rilevazioni_pm2_5['Nome'].values, rilevazioni_pm2_5['Comune'].values))
# Stazioni e rileva PM2.5
for s,c in stazioni_pm2_5_comuni_nomi:
    comune = c.strip()
    if len(comuni[comuni['COMUNE'] == comune]['PRO_COM'].values) != 0:
        if stazioni[stazioni['nome'] == s].shape[0] == 0:
            id_comune = comuni[comuni['COMUNE'] == comune]['PRO_COM'].values[0]
            lat, long = get_lat_long(s, comune)
            row = [s, lat, long, id_comune]
            stazioni.loc[len(stazioni.index)] = row

        row = [s, 'PM2.5']
        rileva.loc[len(rileva.index)] = row

In [13]:
stazioni_pm10_comuni_nomi = set(zip(rilevazioni_pm10['Nome'].values, rilevazioni_pm10['Comune'].values))
# Stazioni e rileva PM10
for s,c in stazioni_pm10_comuni_nomi:
    comune = c.strip()
    if len(comuni[comuni['COMUNE'] == comune]['PRO_COM'].values) != 0:
        if stazioni[stazioni['nome'] == s].shape[0] == 0:
            id_comune = comuni[comuni['COMUNE'] == comune]['PRO_COM'].values[0]
            lat, long = get_lat_long(s, comune)
            row = [s, lat, long, id_comune]
            stazioni.loc[len(stazioni.index)] = row

        row = [s, 'PM10']
        rileva.loc[len(rileva.index)] = row

In [14]:
rileva.head()

,nome_stazione,sigla_indicatore
0,I.T.C. La Porta,PM2.5
1,Mezzo Mobile Via Trani,PM2.5
2,Casardi,PM2.5
3,Terminal Passeggeri,PM2.5
4,I.T.C. De Castro,PM2.5


In [15]:
stazioni.head()

,nome,lat,long,id_comune
0,I.T.C. La Porta,40.1742781,18.1678388,75029
1,Mezzo Mobile Via Trani,41.3214976,16.2868696,110002
2,Casardi,41.3109177,16.2672775,110002
3,Terminal Passeggeri,40.6395295,17.945526,74001
4,I.T.C. De Castro,40.1184626,18.2988529,75039


In [16]:
nomi_stazioni = set(stazioni['nome'])

In [17]:
for i in range(rilevazioni_pm2_5.shape[0]):
    nome_stazione = rilevazioni_pm2_5.loc[i, 'Nome']
    if nome_stazione in nomi_stazioni:
        data = rilevazioni_pm2_5.loc[i, 'Data'][:10]
        valore = rilevazioni_pm2_5.loc[i, 'Valore']
        sigla_indicatore = rilevazioni_pm2_5.loc[i, 'Sigla']
        row = [data, valore, sigla_indicatore, nome_stazione]
        rilevazioni.loc[len(rilevazioni.index)] = row

In [18]:
for i in range(rilevazioni_pm10.shape[0]):
    nome_stazione = rilevazioni_pm10.loc[i, 'Nome']
    if nome_stazione in nomi_stazioni:
        data = rilevazioni_pm10.loc[i, 'Data'][:10]
        valore = rilevazioni_pm10.loc[i, 'Valore']
        sigla_indicatore = rilevazioni_pm10.loc[i, 'Sigla']
        row = [data, valore, sigla_indicatore, nome_stazione]
        rilevazioni.loc[len(rilevazioni.index)] = row

In [19]:
rilevazioni['valore'] = rilevazioni['valore'].str.replace(',','.')
rilevazioni['valore'] = rilevazioni['valore'].astype(float)

In [20]:
rilevazioni[rilevazioni['valore'] == 49.5]

,data_rilevazione,valore,sigla_indicatore,nome_stazione
72333,2019-02-19,49.5,PM10,Stazione


In [21]:
rilevazioni.head()

,data_rilevazione,valore,sigla_indicatore,nome_stazione
0,2019-01-01,7.0,PM2.5,Via Santeramo
1,2019-01-02,8.0,PM2.5,Via Santeramo
2,2019-01-03,4.0,PM2.5,Via Santeramo
3,2019-01-04,6.0,PM2.5,Via Santeramo
4,2019-01-05,8.0,PM2.5,Via Santeramo


In [22]:
stazioni.to_csv('../dataset/stazioni/stazioni_finale.csv', index=False)
rilevazioni.to_csv('../dataset/rilevazioni/rilevazioni_finale.csv', index=False)
rileva.to_csv('../dataset/rileva/rileva_finale.csv', index=False)